In [1]:
import pandas as pd
import numpy as np
import os
import re

In [2]:
# Создаём папку для выходных файлов. Если таковая уже имеется, то очищаем ее от старых файлов.
if os.path.exists("Output"):
    for file in os.listdir("Output/"):
        os.remove("Output/" + file)    
else:        
    os.mkdir("Output")

In [3]:
# Cчитаем количество строк, отведённых под тэги [header section]
def Skip_string (infile):
    count = 0
    with open (infile) as f:
        for line in f:
            if line.startswith("@"):
                count = count + 1
            else:
                break
    return count

In [4]:
# Функция, получающая таблицу формата Bedgraph из листа, содержащего информацию о покрытии каждого элемента
# chr 10, [1 1 1 1 3 3 0 0 4 4 4] (здесь первый элемент имеет нулевой индекс)
#          _______ 
#              |            
#  >> [10 0 4 4]   ___
#                   |
#          [10 4 6 2]      ____
#                           |
#                  [10 8 11 3]

def Make_bedgraph(cover_list):
    
    flg = 0 # необходим для пропуска баз с нулевым покрытием, эти регионы не учитываются в bedgraph файле
    
    # отдельно считаем покрытие для первой базы из исходного файла 
    if cover_list[0]!= 0:
        bed_list[0].append(chrom)
        bed_list[1].append(0)
        bed_list[3].append(cover_list[0])
        flg = 1
        
    for i in range(1, len(cover_list)):
        #print(i)
        if cover_list[i] != cover_list[i-1]: 
            if flg!=0:
                bed_list[2].append(i)  # добавляем конец предыдущего полуинтервала покрытия = началу нового отрезка покрытия. 
            if cover_list[i]!= 0:  
                bed_list[0].append(chrom)
                bed_list[1].append(i)
                bed_list[3].append(np.int(cover_list[i]))
                flg = 1
            if cover_list[i] == 0:
                flg = 0
                
    return(bed_list)   

In [5]:
in_dir = "Input/"
read_len = 300 # Default = 300

for file in os.listdir(in_dir): # Ведём рассчеты для всех файлов, помещенных в папку Input
    print("File: ", file)
    bed_list = [[]*50 for x in range(4)]
    tags_num = Skip_string(in_dir + file) 
    bam = pd.read_csv(in_dir + file, delimiter = "\t", usecols = [2, 3, 5], skiprows = tags_num, names = ["chr", "coord", "CIGAR"])
    bam = bam.sort_values(by = ["chr", "coord"]) # cортируем данные сначала по хромосомам, затем - в пределах каждой из хромосом
    bam = bam.reset_index(drop = True) # заново нумеруем таблицу

    
    chroms = bam.chr.unique() # получаем список хромосом из .sam файла
    print("Chromosomes: ", chroms)

    
    for chrom in chroms:
        
        print("Calculating chromosome {}...".format(chrom))
        bam_for_chr = bam[bam['chr']== chrom]['coord']
        bam_chr_indices = bam_for_chr.index.tolist()
        #print(bam_chr_indices)
        
        
        coverage_all = np.zeros((max(bam_for_chr) + read_len)) # cоздаем лист, который затем заполним информацией о покрытии каждого элемента

        
#         M alignment match (can be a sequence match or mismatch)
#         I insertion to the reference
#         D deletion from the reference
#         N skipped region from the reference
#         S soft clipping (clipped sequences present in SEQ)
#         H hard clipping (clipped sequences NOT present in SEQ)
#         P padding (silent deletion from padded reference)

    
        for i in bam_chr_indices:
            
            # Распознавание CIGAR операторов
            cigar = re.findall(r'(\d+)([A-Z]{1})', bam.CIGAR[i])   #cigar - лист вида: [('13', 'S'), ('138', 'M')]  
            # print(cigar)
            cursor = bam.coord[i]
            
            for c in cigar:
                if c[1] == "M" or c[1] == "=" or c[1] == "X":  # обрабатываем части ридов, которые заматчились
                    coord_start = cursor
                    coord_end = coord_start + int(c[0])
                    cursor = coord_end
                    for i in range(coord_start-1, coord_end-1):
                         coverage_all[i] += 1  
                elif c[1] == "D" or c[1] == "N" or c[1] == "P":
                    cursor = cursor + int(c[0])                

        bed_list = Make_bedgraph(coverage_all)
        
    bed_df = pd.DataFrame(list(zip(*bed_list)),  columns = ["chr", "start", "end", "value"]) 
    print(bed_df[:40])

    # Cохраняем файл 
    file = file.split(".")[0]
    bed_df.to_csv("Output/{}.bedgraph".format(file), sep = "\t", index = False, header = False)
            

File:  for_test.sam
Chromosomes:  ['III' 'IV' 'V']
Calculating chromosome III...
Calculating chromosome IV...
Calculating chromosome V...
    chr  start    end  value
0   III   2555   2655      1
1   III   3100   3200      1
2   III   6790   6890      1
3   III   7234   7334      1
4   III  21931  22031      1
5   III  22351  22451      1
6   III  25570  25670      1
7   III  25996  26096      1
8   III  27361  27461      1
9   III  27756  27856      1
10  III  36606  36706      1
11  III  36967  37067      1
12  III  48187  48287      1
13  III  48499  48599      1
14  III  50455  50555      1
15  III  50788  50888      1
16  III  52456  52556      1
17  III  52857  52957      1
18  III  57527  57627      1
19  III  57959  58059      1
20  III  60423  60523      1
21  III  60838  60938      1
22  III  61596  61696      1
23  III  61894  61994      1
24  III  65328  65428      1
25  III  65767  65867      1
26  III  72590  72690      1
27  III  73047  73147      1
28  III  76940  77040